In [6]:
# Importing pcharmm libraries

import pycharmm
import pycharmm.psf as psf
import pycharmm.read as read
import pycharmm.write as write
import pycharmm.settings as settings
import pycharmm.lingo as lingo

from pycharmm.lib import charmm as libcharmm

In [7]:
# Load the topology files using pyCHARMM  
read.stream('toppar.str')

  
 CHARMM>     stream toppar.str
 VOPEN> Attempting to open::toppar.str::
 OPNLGU> Unit 99 opened for READONLY access to toppar.str

                    INPUT STREAM SWITCHING TO UNIT    99
 RDTITL> * GENERATED BY CHARMM-GUI (HTTP://WWW.CHARMM-GUI.ORG) V3.7 ON AUG, 07. 2023. JOBID=9141926248
 RDTITL> * STREAM FILE FOR TOPOLOGY AND PARAMETER READING
 RDTITL> *
 Parameter: IN1 <- "" <empty>
  
 CHARMM>     
  
 CHARMM>    ! protein topology and parameter
 CHARMM>    open read card unit 10 name toppar/top_all36_prot.rtf
 OPNLGU> Unit already open. The old file will be closed first.
 VCLOSE: Closing unit   10 with status "KEEP"
 VOPEN> Attempting to open::toppar/top_all36_prot.rtf::
 OPNLGU> Unit 10 opened for READONLY access to toppar/top_all36_prot.rtf
  
 CHARMM>    read  rtf card unit 10
 MAINIO> Residue topology file being read from unit  10.
 TITLE> *>>>>>>>>CHARMM36 ALL-HYDROGEN TOPOLOGY FILE FOR PROTEINS <<<<<<
 TITLE> *>>>>> INCLUDES PHI, PSI CROSS TERM MAP (CMAP) CORRECTION <<<<

In [3]:
# Load the psf file using psf_card in pyCHARMM

read.psf_card('traj.psf')
psf.get_nres()

  
 CHARMM>     read psf card -
 CHARMM>     name traj.psf
 VOPEN> Attempting to open::traj.psf::
 MAINIO> Protein structure file being read from unit  91.
 PSFRD2> Reading PSF in the expanded format.
 TITLE>  *PSF FILE
 TITLE>  *  DATE:    11/11/22     13:27:48      CREATED BY USER: raafik
 TITLE>  *
 PSFSUM> PSF modified: NONBOND lists and IMAGE atoms cleared.
 PSFSUM> Summary of the structure file counters :
         Number of segments      =        1   Number of residues   =      214
         Number of atoms         =     3342   Number of groups     =      982
         Number of bonds         =     3366   Number of angles     =     6125
         Number of dihedrals     =     8925   Number of impropers  =      565
         Number of cross-terms   =      212
         Number of HB acceptors  =      322   Number of HB donors  =      377
         Number of NB exclusions =        0   Total charge =   -3.00000
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  


214

In [8]:
#Define the input and output files
#Input: trajectory file
#Output: order parameter file

#remove the old order.dat file
import os
if os.path.exists('order.dat'):
    os.remove('order.dat')

indcd = 'traj.dcd' 
outdat = 'norm.dat'


#CHARMM units are used to read and write files
pycharmm.lingo.charmm_script('open unit 21 write form name ' + outdat ) 
pycharmm.lingo.charmm_script('open unit 62 read unform name ' + indcd )

#Print Trajectory Information
pycharmm.lingo.charmm_script('TRAJectory QUERy UNIT 62') 

  
 CHARMM>     open unit 21 write form name norm.dat
 VOPEN> Attempting to open::norm.dat::
 OPNLGU> Unit 21 opened for WRITE access to norm.dat
  
  
 CHARMM>     open unit 62 read unform name traj.dcd
 OPNLGU> Unit already open. The old file will be closed first.
 VCLOSE: Closing unit   62 with status "KEEP"
 VOPEN> Attempting to open::traj.dcd::
 OPNLGU> Unit 62 opened for READONLY access to traj.dcd
  
  
 CHARMM>     TRAJectory QUERy UNIT 62

 READING TRAJECTORY FROM UNIT  62
   NUMBER OF COORDINATE SETS IN FILE:        1125
   NUMBER OF PREVIOUS DYNAMICS STEPS:           0
   FREQUENCY FOR SAVING COORDINATES:            1
   NUMBER OF STEPS FOR CREATION RUN:         1125

   NUMBER OF DEGREES OF FREEDOM:                0
   NUMBER OF FIXED ATOMS:                       0
   THE INTEGRATION TIME STEP (PS):      0.0489
   THE FILE CONTAINS CRYSTAL UNIT CELL DATA
   THE FILE DOES NOT CONTAIN 4-D DATA
  


1

In [9]:
#CHARMM STtream File Generation for NMR Module, Set options for NMR module
pca_stream = '''* NMR Order Parameter Calculation       
*

! this could be initially read crd as well, (wmain coord array)

COOR DYNA - 
FIRSTU 62 -
NUNIT 1 -
BEGIN 1 -
SKIP 1 -
STOP 1124  -
SELECT ALL END -
ORIENT SELECT ALL END


VIBRAN NMODE 50
QUASI -
NADD 25 -     ! skip first NADD modes
NFREQ 2 -     ! take 2 modes
FIRSTU 62 -
NUNIT 1 -
SELECT TYPE CA END -
NORM -
NOTP

write NORM MODE 1 unit 21 card
END

RETURN
'''
with open('pca.str', 'w') as f:
    f.write(pca_stream)

# Running NMR Module via read option in pyCHARMM
read.stream('pca.str')

  
 CHARMM>     stream pca.str
 VOPEN> Attempting to open::pca.str::
 OPNLGU> Unit 99 opened for READONLY access to pca.str

                    INPUT STREAM SWITCHING TO UNIT    99
 RDTITL> * NMR ORDER PARAMETER CALCULATION
 RDTITL> *
 Parameter: IN1 <- "" <empty>
  
 CHARMM>     
  
 CHARMM>    ! this could be initially read crd as well, (wmain coord array)
 CHARMM>     
  
 CHARMM>    COOR DYNA -
 CHARMM>    FIRSTU 62 -
 CHARMM>    NUNIT 1 -
 CHARMM>    BEGIN 1 -
 CHARMM>    SKIP 1 -
 CHARMM>    STOP 1124  -
 CHARMM>    SELECT ALL END -
 CHARMM>    ORIENT SELECT ALL END
 SELRPN>   3342 atoms have been selected out of   3342
 SELRPN>   3342 atoms have been selected out of   3342
      * ALL COORDINATE FRAMES WILL BE REORIENTED WITH  RESPECT TO THE COMPARISON SET *

 READING TRAJECTORY FROM UNIT  62
   NUMBER OF COORDINATE SETS IN FILE:        1125
   NUMBER OF PREVIOUS DYNAMICS STEPS:           0
   FREQUENCY FOR SAVING COORDINATES:            1
   NUMBER OF STEPS FOR CREATION RUN:  

: 

In [ ]:
#Plotting the RESI vs <S2> plot from the order.dat file

import numpy as np
import matplotlib.pyplot as plt

# Load data from 'order.dat', skipping the header row
data = np.genfromtxt('order.dat', skip_header=1, usecols=(0, 6), dtype=float, filling_values=np.nan)

# Remove rows with NaN values (rows that had non-numeric data)
data = data[~np.isnan(data).any(axis=1)]

# Extract 'resi' and '<S2>' columns from the data
resi_values = data[:, 0]  # Assuming 'resi' is in the first column (index 0)
s2_values = data[:, 1]     # Assuming '<s2>' is in the second column (index 1)

# Plotting using matplotlib
plt.plot(resi_values, s2_values, marker='o')
plt.xlabel('RESI')
plt.ylabel('<S2>')
plt.title('RESI vs <S2> Plot')
plt.grid(True)
plt.show()